# Question 2 - Analysis 1 
- From Article Search API, collect data and store it as json file.
- analyse the data for each year
- from the articles identify the popular sport for each year and month.

In [1]:
import os
import glob
from email.parser import Parser
import email.utils
import csv
import json
from datetime import datetime
import collections 

In [2]:
data_path = os.path.expanduser('~') 
data_path = os.path.join(data_path, 'midterm')
data_path = os.path.join(data_path, 'data')
path = os.path.join(data_path,'Sports')

In [3]:
files = glob.glob(path+ '\\' + '*' + '.json')
#print(files)

In [4]:
sport_data ={}
for file in files:
    with open(file, "r") as f:
        data = json.load(f) #load the json data in dictionary
        #get the value of tags 
        response = data.get('response') 
        docs = response.get('docs')
        for x in docs:
            article_date = x.get('pub_date') #get the article date
            year = article_date[:4]          #substring article date to get year
            month = article_date[5:7]        #substring artivle date to get month
            section_name = x.get('section_name') 
            
            #process only those articles from Sports sections
            
            if section_name == 'Sports':
                subsection_name = x.get('subsection_name')
                if subsection_name is not None:                                 #check if sucsection name is not empty
                    subsection_name = subsection_name.replace('Pro ','')
                    subsection_name = subsection_name.replace('College ','')
                    if year not in sport_data:                                  #for each year and month, count the number of articles published for each sport
                        sport_data[year] = {}                                   #create empty dictionary to store month,sport and count
                        sport_data[year][month] = {}                            #dictionary that stores sport and count
                        sport_data[year][month][subsection_name] =1             #assign the count as 1 as this is the first article for this year
                    else:
                        if month not in sport_data[year]:
                            sport_data[year][month] = {}
                            sport_data[year][month][subsection_name] =1
                        else: 
                            if subsection_name not in sport_data[year][month]:
                                sport_data[year][month][subsection_name] =1
                            else:
                                sport_data[year][month][subsection_name] += 1
                            
#print(sport_data)

In [5]:
OutputFile = open ('PopularSport.csv', 'wt', newline = '' , encoding='utf-8') #create and open CSV file in wirte mode
recordwriter = csv.writer(OutputFile)                                        
recordwriter.writerow(['Year', 'Month' , 'Sports Name' , 'No of Articles Publised']) # write header to csv file

#sort the dictionaly sport_data which has count for each sport for every month of each year
sortedYear = sorted(sport_data)             # sort the dictionary based on key
for year in sortedYear:
    #print(year)
    listOfMonths = []
    listOfMonths = sorted(sport_data[year])
    #print(listOfMonths)
    for month in listOfMonths:
        listOfSports = []
        listOfSports = sorted(sport_data[year][month], key=sport_data[year][month].__getitem__ , reverse= True)
        for sport in listOfSports:
            recordwriter.writerow([year, month , sport, sport_data[year][month][sport] ] ) 
    recordwriter.writerow([''] )  #writes an empty record in csv

OutputFile.close()
